In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import f1_score

%matplotlib inline

seed = 1356
raw_data = pd.read_pickle("domestic_audio_features.pkl")

afs, labels, _ = zip(*[line.rstrip('\n').split('\t') for line in open('DCASE2018-task5-dev.meta/DCASE2018-task5-dev/meta.txt').readlines()])

sces = set(labels)
sce_int_map = {sce:i+1 for i, sce in enumerate(sces)}
int_sce_map = {sce_int_map[i]:i for i in sce_int_map.keys()}

# split data such that each label is equally represented
bal_data = raw_data.groupby("label")
bal_data = bal_data.apply(lambda x: x.sample(bal_data.size().min(), random_state=seed).reset_index(drop=True))
y = bal_data["label"]
X = bal_data.drop("label", axis=1)

# split data into stratified folds for cross validation
# training, validation set is in a ratio of 3:1
skf = StratifiedKFold(n_splits=4, random_state=seed)

# generate folds and pre-process the data
# std_train, minmax_train, y_train, std_test, minmax_test, y_test
folds = []
for train, test in skf.split(X, y):
    X_train, X_test = X.values[train], X.values[test]
    y_train, y_test = y.values[train], y.values[test]
    
    # convert y into one-hot-encode (ie. [x x x x x x x x x] where x is 1 if index = label and 0 otherwise)
    y_train = np.array([[1 if c == label - 1 else 0 for c in range(len(sces))] for label in y_train])
    y_test = np.array([[1 if c == label - 1 else 0 for c in range(len(sces))] for label in y_test])


    std_scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_std = std_scaler.transform(X_train)
    X_test_std = std_scaler.transform(X_test)

    minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
    X_train_minmax = minmax_scaler.transform(X_train)
    X_test_minmax = minmax_scaler.transform(X_test)
    folds.append([X_train_std, X_train_minmax, y_train, X_test_std, X_test_minmax, y_test])

In [2]:
# 50 features, 9 labels/classes

import tensorflow as tf

n_features = 50
n_classes = 9

sd = 1 / np.sqrt(n_features)

tfX = tf.placeholder(tf.float32, [None, n_features])
tfY = tf.placeholder(tf.float32, [None, n_classes])

def ff_neural_model(data, hidden_units = (200, 200)):
    
    temp_units = (n_features,) + hidden_units
    
    hidden_layers = [{'weights':tf.Variable(tf.random_normal([temp_units[i], units], seed=seed, mean=0, stddev=sd)),
                      'biases':tf.Variable(tf.zeros([units]))} for i, units in enumerate(hidden_units)]
    
    output_layer = {'weights':tf.Variable(tf.random_normal([hidden_units[-1], n_classes], seed=seed, mean=0, stddev=sd)),
                    'biases':tf.Variable(tf.zeros([n_classes]))}
    
    
    res = data
    for layer in hidden_layers:
        res = tf.add(tf.matmul(res, layer['weights']), layer['biases'])
        res = tf.nn.relu(res)
    
    output = tf.matmul(res, output_layer['weights']) + output_layer['biases']
    
    return output

def train_neural_network(x, xtrain, ytrain, xtest, ytest, batch_size = -1, epochs = 500, show_graphs = False, hidden_units = (200, 200), verbose = False, show_accuracy = False, show_checkpoints = False):
    pred = ff_neural_model(x, hidden_units)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=tfY))
    #cost = -tf.reduce_sum(tfY * tf.log(pred))
    optimiser = tf.train.AdamOptimizer().minimize(cost)
    
    training_epochs = epochs
    
    with tf.Session() as sess:
        
        training_acc, training_loss = [], []
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(training_epochs):
            epoch_loss = 0
            index = 0
            while index < len(xtrain):
                if batch_size == -1:
                    batch_x = np.array(xtrain)
                    batch_y = np.array(ytrain)
                    
                    _, c = sess.run([optimiser, cost], feed_dict={x: batch_x, tfY: batch_y})
                    epoch_loss += c
                    break
                    
                batch_x = np.array(xtrain[index:index+batch_size])
                batch_y = np.array(ytrain[index:index+batch_size])
                
                _, c = sess.run([optimiser, cost], feed_dict={x: batch_x, tfY: batch_y})
                epoch_loss += c
                index += batch_size
            
            if show_accuracy:
                correct = tf.equal(tf.argmax(pred, 1), tf.argmax(tfY, 1))
                accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)).eval({x: xtrain, tfY: ytrain})
                training_acc.append(accuracy)
                
            training_loss.append(epoch_loss)
            
            if show_checkpoints:
                if epoch+1 in [int(.25 * epochs), int(.5 * epochs), int(.75 * epochs)]:
                    print("{}% complete.".format(int(100*(epoch+1)/epochs)))
            
            if verbose:
                print("Epoch {} completed out of {}. Loss: {}.".format(epoch + 1, training_epochs, epoch_loss), end=' ')
                if show_accuracy:
                    print("Accuracy: {}.".format(accuracy))
                else:
                    print()
        
        correct = tf.equal(tf.argmax(pred, 1), tf.argmax(tfY, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)).eval({x: xtest, tfY: ytest})
        
        y_pred = sess.run(tf.argmax(pred, 1), feed_dict={x: xtest})
        y_true = sess.run(tf.argmax(ytest, 1))
        
        if show_graphs:
            x_axis = np.arange(1,epochs+1,1)
            if show_accuracy:
                plt.plot(x_axis, 10 * np.array(training_acc), label='Training accuracy (x10)')
            plt.plot(x_axis, np.log(training_loss), label='Loss (ln scale)')
            plt.legend()
        
        # print("Accuracy: {}".format(accuracy))
        return accuracy, f1_score(y_true, y_pred, average='macro')

c:\users\jerem_000\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, show_checkpoints=True)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, show_checkpoints=True)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8783721923828125. F1: 0.8779266275720157

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8614540696144104. F1: 0.8589937276009603

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8888888955116272. F1: 0.8885341339188535

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8760859370231628. F1: 0.8719753138556797

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8888888955116272. F1: 0.8882296998270252

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8683127760887146. F1: 0.8671914987409043

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.873799741268158. F1: 0.8729218971918612

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8637402653694153. F1: 0.85865225002

In [14]:
batch_size = 100

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, show_checkpoints=True, batch_size=batch_size)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, show_checkpoints=True, batch_size=batch_size)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8770004510879517. F1: 0.8775134677927043

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.7032464742660522. F1: 0.697169942282119

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8856881856918335. F1: 0.8844771347720397

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8673982620239258. F1: 0.8657976469165626

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8774576783180237. F1: 0.8774224898397374

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.7540009021759033. F1: 0.7485678349543143

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8838591575622559. F1: 0.8833238520509522

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8221307992935181. F1: 0.82310635402

In [15]:
hidden_units = (100,)

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8673982620239258. F1: 0.8666139668948829

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8267032504081726. F1: 0.8212065913630483

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8733424544334412. F1: 0.8708492375032412

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8390489220619202. F1: 0.8303182456539931

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.87425696849823. F1: 0.8724372035672314

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8427069187164307. F1: 0.8368767988908016

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8587105870246887. F1: 0.8556888402082723

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8303612470626831. F1: 0.821720727761

In [16]:
hidden_units = (300,200,100)

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8783721923828125. F1: 0.8788761898170145

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8372199535369873. F1: 0.83592276306514

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8934613466262817. F1: 0.8935297071769526

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8573388457298279. F1: 0.8534133312247358

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8724279999732971. F1: 0.8726858248142645

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.798811137676239. F1: 0.8043254712573932

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8751714825630188. F1: 0.8748168099157994

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8504801392555237. F1: 0.8455538928310

In [17]:
hidden_units = (30,30)

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8449931144714355. F1: 0.8431558795353928

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8074988722801208. F1: 0.8024760696456834

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8582533001899719. F1: 0.8555766763789237

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8385916948318481. F1: 0.8263713039180124

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.840877890586853. F1: 0.8387813054214986

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8116140961647034. F1: 0.810572961670882

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8372199535369873. F1: 0.8323723720832561

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8267032504081726. F1: 0.817242606713

In [19]:
hidden_units = (30,)

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = 100, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8555098176002502. F1: 0.8530047280081372

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8116140961647034. F1: 0.8052129186496748

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8646547794342041. F1: 0.8609222902614362

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8180155754089355. F1: 0.8082185862529153

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8646547794342041. F1: 0.8620974128917077

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8212162852287292. F1: 0.8130844309509822

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8550525903701782. F1: 0.8517889402051542

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8166437745094299. F1: 0.8079964313

In [20]:
hidden_units = (50, 50)
batch_size = 64

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = batch_size, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = batch_size, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8459076285362244. F1: 0.8462269586913286

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8353909254074097. F1: 0.8312388684622563

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8637402653694153. F1: 0.8618886825006729

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8349336981773376. F1: 0.8310005332440329

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8376771807670593. F1: 0.8356741256750165

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8353909254074097. F1: 0.8312705890261576

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8491083383560181. F1: 0.8479915194137353

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8262460231781006. F1: 0.8210639469

In [4]:
hidden_units = (200,200)
batch_size = 64

for i,(train_std, train_minmax, train_y, test_std, test_minmax, test_y) in enumerate(folds):
    std_acc, minmax_acc, std_f1, minmax_f1 = [], [], [], []
    print("Standard preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_std, train_y, test_std, test_y, batch_size = batch_size, show_checkpoints=True, hidden_units=hidden_units)
    std_acc += [acc]
    std_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
    print("Minmax preprocessing. Fold {}".format(i+1))
    acc, f1 = train_neural_network(tfX, train_minmax, train_y, test_minmax, test_y, batch_size = batch_size, show_checkpoints=True, hidden_units=hidden_units)
    minmax_acc += [acc]
    minmax_f1 += [f1]
    print("Accuracy: {}. F1: {}".format(acc, f1))
    print()
print("Standard accuracy: {}. Minmax accuracy: {}.".format(np.mean(std_acc), np.mean(minmax_acc)))
print("Standard f1: {}. Minmax f1: {}.".format(np.mean(std_f1), np.mean(minmax_f1)))

Standard preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8555098176002502. F1: 0.8551502301924327

Minmax preprocessing. Fold 1
25% complete.
50% complete.
75% complete.
Accuracy: 0.8029263615608215. F1: 0.7986002500596542

Standard preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.8907178640365601. F1: 0.8912407972473478

Minmax preprocessing. Fold 2
25% complete.
50% complete.
75% complete.
Accuracy: 0.855967104434967. F1: 0.852060841721626

Standard preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.8692272305488586. F1: 0.8691505590941262

Minmax preprocessing. Fold 3
25% complete.
50% complete.
75% complete.
Accuracy: 0.7764060497283936. F1: 0.7808524589198673

Standard preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8765432238578796. F1: 0.8757781646984388

Minmax preprocessing. Fold 4
25% complete.
50% complete.
75% complete.
Accuracy: 0.8353909254074097. F1: 0.832915082179